<a href="https://colab.research.google.com/github/liorhirsch/Kaggle-Landmark-ML-Challange/blob/master/Custom%20NN/Basic_Custom_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload data from google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Build training data


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from joblib import Parallel, delayed
import multiprocessing

**Augmentated Data Generation**

Creates images generators both for the test and the training data, to create additional data

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen=ImageDataGenerator(
                              rotation_range=0,
                              width_shift_range=0,
                               height_shift_range=0,
                               shear_range=0,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies

test_datagen = ImageDataGenerator(
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.8, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5]) #included in our dependencies


 Uses the data as a stream object to enable batching on it instead of loading them all together to the ram using the ImageDataGenerator

In [5]:
image_size = (100,100)
batch_size = 20

training_data_dir = 'drive/My Drive/photos'
validation_data_dir = 'drive/My Drive/test'

train_generator=train_datagen.flow_from_directory(training_data_dir,
                                                 target_size=image_size,
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_generator = test_datagen.flow_from_directory(
                                        validation_data_dir,
                                         target_size=image_size,
                                         color_mode='rgb',
                                        class_mode = "categorical")

Found 67559 images belonging to 100 classes.
Found 18780 images belonging to 100 classes.


In [0]:
x_batch, y_batch = next(train_generator)
# print(x_batch) # RGB 2d array
# print(y_batch) # The label (class)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20, 20))
for i in range(10):
    plt.subplot(5,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image = x_batch[i]
    image= image.astype(np.uint8)
    plt.imshow(image)

# Build The Network

In [0]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv3D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

**Normalize the images.**

In [0]:
x_batch = x_batch/255.0
# print(x_batch)

**Build the NN**

In [0]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = x_batch.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatening the data
model.add(Flatten())
model.add(Dense(150))
model.add(Activation("relu"))

#output layer
model.add(Dense(100))
model.add(Activation('softmax'))

In [11]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
__________

We use Adam Optimizer to...

In [0]:
adamOptimizer = tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss = 'sparse_categorical_crossentropy', 
             optimizer='adam',
             metrics=['accuracy'])

# model.compile(loss='categorical_crossentropy', optimizer='adam',\
#  metrics=['accuracy'])

We convert the generator's output from binary array that contains 1 in the related class, to array contains the indexes of the classes for each picture in the batch

In [13]:
y=[a.tolist().index(1) for a in y_batch];
print(y)

[28, 10, 55, 77, 26, 0, 16, 33, 9, 50, 32, 3, 92, 7, 26, 15, 25, 64, 22, 76]


# Train

Let us configure a loss function and optimizer.

The generator gives us in y a vector contains match percentages for each label, so we want to change it to show 0 or 1 using caregorical_crossentropy

In [0]:
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

we calculate the amount of batches in an ephoch (For fitting the model)

In [15]:
step_size_train=train_generator.n//train_generator.batch_size
print(step_size_train)

3377


We would like to observe the loss function values throught the training process. 
This is why we create a callback to store the loss and accuracy function values for each batch :

In [0]:
class LossAccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('acc'))
        
history = LossAccHistory()

Fix for the error : 
OSError: image file is truncated (14 bytes not processed)

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True 

#https://stackoverflow.com/questions/12984426/python-pil-ioerror-image-file-truncated-with-big-images

Now we train the model with fit_generator instead of regular fit.

In [0]:
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train, # step=how much batches in one epoc
                   validation_data = validation_generator,
                   epochs=5
                   ,callbacks = [history])

Epoch 1/5
  17/3377 [..............................] - ETA: 6:17:06 - loss: 15.5966 - acc: 0.0324

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


1345/3377 [==========>...................] - ETA: 3:52:00 - loss: 15.7828 - acc: 0.0207

In [0]:
import matplotlib.pyplot as plt
x = range(0, len(history.losses))
plt.plot(x, history.losses)
plt.plot(x, history.accuracy)
plt.xlabel('batches')
plt.title('Loss and Accuracy')
plt.legend(['losses', 'accuracy'])

# Save the model

Create results directory if it doesn't exist

In [0]:
import os
path_results="drive/My Drive/results/zolantz/"
if not os.path.exists(path):
    os.makedirs(path)

In [0]:
!pip install h5py

In [0]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# Load the model from h5 file

load the model from the saved file

In [0]:
from tensorflow.keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [0]:
train_loss, train_acc = model.evaluate(X,y)

In [0]:
print("loss: ",train_loss, "acc: ",  train_acc)

# Test the model on the TestSet

Lets evaluate our model on the test set :


In [0]:
train_loss, train_acc = model.evaluate(x_batch,y)


In [0]:
print(train_loss)
print(train_acc)